In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
import re

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.mohfw.gov.in/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[-1]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:

# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
df_bs.drop('S. No.', axis=1, inplace=True)

# there are 36 states+UT in India
df_bs.head(36)

df_bs=df_bs.rename(columns={"Name of State / UT": "State"}, errors="raise")

In [6]:

# date-time information
# =====================

# today's date
now  = datetime.now()
# format date to month-day-year
df_bs['Date'] = now.strftime("%m/%d/%Y") 

# add 'Date' column to dataframe
df_bs['Date'] = pd.to_datetime(df_bs['Date'], format='%m/%d/%Y')

# df_bs.head(36)

In [7]:

# remove extra characters from 'Name of State/UT' column
df_bs['State'] = df_bs['State'].str.replace('#', '')

In [8]:
# latitude and longitude information
# ==================================

# latitude of the states
lat = {'Delhi':28.7041, 'Haryana':29.0588, 'Kerala':10.8505, 'Rajasthan':27.0238,
       'Telengana':18.1124, 'Uttar Pradesh':26.8467, 'Ladakh':34.2996, 'Tamil Nadu':11.1271,
       'Jammu and Kashmir':33.7782, 'Punjab':31.1471, 'Karnataka':15.3173, 'Maharashtra':19.7515,
       'Andhra Pradesh':15.9129, 'Odisha':20.9517, 'Uttarakhand':30.0668, 'West Bengal':22.9868, 
       'Puducherry': 11.9416, 'Chandigarh': 30.7333, 'Chhattisgarh':21.2787, 'Gujarat': 22.2587, 
       'Himachal Pradesh': 31.1048, 'Madhya Pradesh': 22.9734, 'Bihar': 25.0961, 'Manipur':24.6637, 
       'Mizoram':23.1645, 'Goa': 15.2993, 'Andaman and Nicobar Islands': 11.7401, 'Assam' : 26.2006, 
       'Jharkhand': 23.6102, 'Arunachal Pradesh': 28.2180, 'Tripura': 23.9408, 'Nagaland': 26.1584, 
       'Meghalaya' : 25.4670, 'Dadar Nagar Haveli' : 20.1809, 'Sikkim':27.5330}

# longitude of the states
long = {'Delhi':77.1025, 'Haryana':76.0856, 'Kerala':76.2711, 'Rajasthan':74.2179,
        'Telengana':79.0193, 'Uttar Pradesh':80.9462, 'Ladakh':78.2932, 'Tamil Nadu':78.6569,
        'Jammu and Kashmir':76.5762, 'Punjab':75.3412, 'Karnataka':75.7139, 'Maharashtra':75.7139,
        'Andhra Pradesh':79.7400, 'Odisha':85.0985, 'Uttarakhand':79.0193, 'West Bengal':87.8550, 
        'Puducherry': 79.8083, 'Chandigarh': 76.7794, 'Chhattisgarh':81.8661, 'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 'Madhya Pradesh': 78.6569, 'Bihar': 85.3131, 'Manipur':93.9063, 
        'Mizoram':92.9376, 'Goa': 74.1240, 'Andaman and Nicobar Islands': 92.6586, 'Assam' : 92.9376, 
        'Jharkhand': 85.2799, 'Arunachal Pradesh': 94.7278, 'Tripura': 91.9882, 'Nagaland': 94.5624,
        'Meghalaya' : 91.3662, 'Dadar Nagar Haveli' : 73.0169, 'Sikkim':88.5122}

# add latitude column based on 'Name of State / UT' column
df_bs['Latitude'] = df_bs['State'].map(lat)

# add longitude column based on 'Name of State / UT' column
df_bs['Longitude'] = df_bs['State'].map(long)


In [9]:
# read data
# data about number of goverment hospitals and beds in hospital
##df1 = pd.read_csv('https://raw.githubusercontent.com/souroy12/Hospital-Bed-Analysis/master/Number%20of%20Government%20Hospitals%20and%20Beds%20in%20Rural%20and%20Urban%20Areas.csv')
# read data
# data about number of goverment hospitals and beds in hospital
df1 = pd.read_csv('C://Users/skoul2/AnacondaProjects/Covid/datasets_Number of Hospitals and Beds in Public and Private Areas .csv')

In [10]:
combined_df=pd.merge(df_bs, df1, how='left')

In [11]:
combined_df = combined_df.drop(17)

In [12]:
combined_df=combined_df.rename(columns={"Deaths**": "Deaths"}, errors="raise")
combined_df=combined_df.rename(columns={"Total Confirmed cases*": "Confirmed"}, errors="raise")
combined_df=combined_df.rename(columns={"Cured/Discharged/Migrated*": "Cured_Discharged_Migrated"}, errors="raise")
combined_df=combined_df.rename(columns={"Active Cases*": "Active"}, errors="raise")


In [13]:
#convert data types of numerical values into int
combined_df[['Active','Deaths','Confirmed','Cured_Discharged_Migrated']]=combined_df[['Active','Deaths','Confirmed','Cured_Discharged_Migrated']].astype(int) 

In [ ]:
from flask import Flask
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext
import pandas as pd
import requests
# window
app = Flask(__name__)

@app.route('/')
def sms():
    win = tk.Tk()
    win.title("Python GUI")
    # win.resizable(0,0)

    # modify adding label
    aLabel = ttk.Label(win, text="A Label")
    aLabel.grid(column=0, row=0)

    # button click event
    # def clickMe():
    #     action.configure(text='Number saved ' + name.get())
    def callback(eventObject):
        print(numberChosen.get())


    def writeToFile(contact,SConfirmed,SRecovered,SBeds,TConfirmed,TRecovered,TBeds,stateselect):
            url = "https://www.fast2sms.com/dev/bulk"
            payload = "sender_id=COVID19SMS&message= State:All,State Confirmed cases:%s, Recovered Cases:%s,  Available Beds:%s,All Confirmed cases:%s, All Recovered Cases:%s, All Available Beds:%s&language=english&route=p&numbers=%d" %(TConfirmed,TRecovered,TBeds,SConfirmed,SRecovered,SBeds,int((contact)))
            headers = {
            'authorization': "NwTGEmFtzUPD2MeaQKVqp6fcXBhruSdA9kOWCxnIRYyLib7Zv0XdMh4k6Jn0SylVufHcRtzG1eOLBPaU",
            'Content-Type': "application/x-www-form-urlencoded",
            'Cache-Control': "no-cache",
                }
            response = requests.request("POST",url, headers=headers,data=payload)
            print(response.text)

    # text box entry
    ttk.Label(win, text="Contact Number:").grid(column=0, row=0)
    contact = tk.StringVar()
    contactNumber = ttk.Entry(win, width=12, textvariable=contact)
    contactNumber.grid(column=0, row=1)


    # action.configure(state='disabled')
    ###df=pd.read_csv('statelist.csv')
    combineddf1=combined_df['State']
    combineddf2=combined_df['Confirmed']
    combineddf3=combined_df['Cured_Discharged_Migrated']
    combineddf4=combined_df['total_beds']
    data=combineddf1.values.tolist()
    dataConfirmed=combineddf2.values.tolist()
    dataRecovered=combineddf3.values.tolist()
    databeds=combineddf4.values.tolist()
    TConfirmed=sum(int(i) for i in dataConfirmed)

    TRecovered=sum(int(j) for j in dataRecovered)
    Tbeds=sum(int(k) for k in databeds)


    # drop down menu
    ttk.Label(win, text="State").grid(column=1, row=0)
    number = tk.StringVar()
    numberChosen = ttk.Combobox(win, width=12, textvariable=number)
    numberChosen['values'] = data
    numberChosen.grid(column=1, row=1)
    numberChosen.set("All")
    stateselect=numberChosen.bind("<<ComboboxSelected>>", callback)
    stateselect
    ###df111=pd.read_csv('statelist.csv')
    #f111
    dfSelectstate= combined_df[combined_df['State']==stateselect]

    dfSelectstate1=combined_df['State']
    dfSelectstate2=combined_df['Confirmed']
    dfSelectstate3=combined_df['Cured_Discharged_Migrated']
    dfSelectstate4=combined_df['total_beds']
    #rint(dfSelectstate1)

    # button
    action = ttk.Button(win, text="Get Alert!", command= lambda: writeToFile(contact.get(),TConfirmed,TRecovered,Tbeds,TConfirmed,TRecovered,Tbeds,stateselect))
    action.grid(column=2, row=1)


    # direct keyboard input to text box entry
    contactNumber.focus()

    return win.mainloop()

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Goa


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\skoul2\AppData\Local\Continuum\anaconda2\envs\py36\lib\site-packages\urllib3\contrib\pyopenssl.py", line 485, in wrap_socket
    cnx.do_handshake()
  File "C:\Users\skoul2\AppData\Local\Continuum\anaconda2\envs\py36\lib\site-packages\OpenSSL\SSL.py", line 1934, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "C:\Users\skoul2\AppData\Local\Continuum\anaconda2\envs\py36\lib\site-packages\OpenSSL\SSL.py", line 1671, in _raise_ssl_error
    _raise_current_error()
  File "C:\Users\skoul2\AppData\Local\Continuum\anaconda2\envs\py36\lib\site-packages\OpenSSL\_util.py", line 54, in exception_from_error_queue
    raise exception_type(errors)
OpenSSL.SSL.Error: [('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\skoul2\AppData\Local\Continuum\ana